In [1]:
import rebound
import time
import sys
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants as c

# local imports
import heartbeat
import globs
import metasim

In [2]:
def run_sim(filestem='test/test'):
    
    # global variables

    globs.initialise()

    globs.glob_dclo = 1. #CE distance to check in rH
    #dir = 'test/'
    globs.glob_archive = ''
    #globals.glob_names = []
    globs.glob_is_close = False

    sys = metasim.MetaSim(filestem=filestem)
    sys.run_planets()
    if globs.glob_is_close:
        sys.rewind()
        sys.add_moons()
        sys.run_moons()
        sys.analyse()
        sys.make_timeline()

In [ ]:
#Nsys = 1

#for i in range(Nsys):
#    run_sim(filestem=f'test/test{i:04d}')

i = 1
run_sim(filestem=f'test/test{i:04d}')

Simulation test/test0001

Creating new simulation...
Running planets-only simulation...
CE between Planet1 and Planet2 with dist 0.037477301204427174 au at 3574.744857412121 years
<rebound.particle.Particle object at 0x7fccf8cafbc0, m=0.0019091884679386499 x=-1.257951922721229 y=1.561636522344932 z=-0.012232572607888622 vx=-3.879509095603708 vy=-3.4187718820212387 vz=0.4035662412516596>
<rebound.particle.Particle object at 0x7fccf8cafb40, m=0.0019091884679386499 x=-1.2260532457033146 y=1.5419871995587895 z=-0.011269959435258844 vx=-2.443146954392125 vy=-1.3298252591560271 vz=-0.3512326929653351>
CE between Planet2 and Planet1 with dist 0.037477301204427174 au at 3574.744857412121 years
<rebound.particle.Particle object at 0x7fccf8cafc40, m=0.0019091884679386499 x=-1.2260532457033146 y=1.5419871995587895 z=-0.011269959435258844 vx=-2.443146954392125 vy=-1.3298252591560271 vz=-0.3512326929653351>
<rebound.particle.Particle object at 0x7fccf8cafac0, m=0.0019091884679386499 x=-1.2579519227

/Users/alexander/research/circumplanetary_discs/rebound/rebound/rebound/simulation.py:712: RuntimeWarning: File in use for SimulationArchive already exists. Snapshots will be appended.
  warnings.warn(msg[1:], RuntimeWarning)


CE between Planet1 and Planet2 with dist 0.03749975174657276 au at 3574.7434817923804 years
<rebound.particle.Particle object at 0x7fcce8de49c0, m=0.0019091884679386499 x=-1.252478781543692 y=1.5661517726623237 z=-0.012770370450591454 vx=-3.944580722926975 vy=-3.368642222493547 vz=0.3998071023116296>
<rebound.particle.Particle object at 0x7fcce8de4940, m=0.0019091884679386499 x=-1.2224843575249518 y=1.5437305450473386 z=-0.010807092305336208 vx=-2.391946890247061 vy=-1.3624194566730208 vz=-0.34761742222239533>
CE between Planet2 and Planet1 with dist 0.03749975174657276 au at 3574.7434817923804 years
<rebound.particle.Particle object at 0x7fcce8de4bc0, m=0.0019091884679386499 x=-1.2224843575249518 y=1.5437305450473386 z=-0.010807092305336208 vx=-2.391946890247061 vy=-1.3624194566730208 vz=-0.34761742222239533>
<rebound.particle.Particle object at 0x7fcce8de4b40, m=0.0019091884679386499 x=-1.252478781543692 y=1.5661517726623237 z=-0.012770370450591454 vx=-3.944580722926975 vy=-3.3686422